# DTOcean Moorings and Foundation Test

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [4]:
import numpy as np

In [5]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu
from dtocean_core.pipeline import Tree

In [6]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [7]:
# Bring up the logger
start_logging()

2017-05-12 15:03:21,453 - INFO - dtocean_core - Begin logging for dtocean_core.


## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [8]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
pipe_tree = Tree()

## Create a new project

In [9]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2017-05-12 15:03:25,881 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2017-05-12 15:03:25,885 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2017-05-12 15:03:25,891 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [10]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type" 
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Tidal Floating")
my_var.read(new_core, new_project)

2017-05-12 15:03:25,911 - INFO - aneris.control.data - New "device.system_type" data stored with index DNJR8B
2017-05-12 15:03:25,911 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [11]:
project_menu.initiate_pipeline(new_core, new_project)

2017-05-12 15:03:25,934 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2017-05-12 15:03:25,950 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index JBIUPQ
2017-05-12 15:03:25,951 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2017-05-12 15:03:25,957 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2017-05-12 15:03:25,960 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [12]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [13]:
module_name = 'Mooring and Foundations'
module_menu.activate(new_core, new_project, module_name)

## Check the status of the module inputs

In [14]:
mooring_branch = pipe_tree.get_branch(new_core, new_project, 'Mooring and Foundations')
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [15]:
project_menu.initiate_dataflow(new_core, new_project)

2017-05-12 15:03:26,042 - INFO - aneris.control.data - New "hidden.dataflow_active" data stored with index RO42AK
2017-05-12 15:03:26,045 - INFO - aneris.control.simulation - Datastate stored
2017-05-12 15:03:26,065 - INFO - aneris.control.simulation - Datastate with level "modules initial" stored


## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [16]:
%run inputs_wp4.py

generate test data: C:\Users\mtopper\Programming\Python\notebooks\inputs_wp4.pkl


In [17]:
mooring_branch.read_test_data(new_core,
                              new_project,
                              "inputs_wp4.pkl")

2017-05-12 15:03:53,003 - INFO - aneris.control.data - New "farm.max_gamma_100_year" data stored with index GWZ63G
2017-05-12 15:03:53,007 - INFO - aneris.control.data - New "farm.mean_wind_speed_100_year" data stored with index BO55ZR
2017-05-12 15:03:53,012 - INFO - aneris.control.data - New "device.system_profile" data stored with index 25Y1LI
2017-05-12 15:03:53,016 - INFO - aneris.control.data - New "project.substation_cog" data stored with index THVOV5
2017-05-12 15:03:53,020 - INFO - aneris.control.data - New "farm.max_water_level_50_year" data stored with index 5QUT0C
2017-05-12 15:03:53,028 - INFO - aneris.control.data - New "farm.mean_wind_direction_100_year" data stored with index JJ8MTK
2017-05-12 15:03:53,032 - INFO - aneris.control.data - New "device.turbine_diameter" data stored with index 87TAJI
2017-05-12 15:03:53,042 - INFO - aneris.control.data - New "constants.pile_Bm_moment_coefficient" data stored with index E12S4U
2017-05-12 15:03:53,049 - INFO - aneris.control.d

## Check if the module can be executed

In [18]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

True

In [19]:
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [20]:
module_menu.execute_current(new_core, new_project)

2017-05-12 15:03:53,683 - INFO - aneris.control.simulation - Datastate with level "mooring and foundations start" stored
2017-05-12 15:03:56,346 - INFO - dtocean_moorings.main - ===================================================================
2017-05-12 15:03:56,348 - INFO - dtocean_moorings.main - Device ID: device001
2017-05-12 15:03:56,351 - INFO - dtocean_moorings.main - ===================================================================
2017-05-12 15:03:57,413 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2017-05-12 15:03:57,415 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2017-05-12 15:03:57,417 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2017-05-12 15:03:57,421 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2017-05-12 15:03:57,423 - WARNING - dtocean_moorings.co

c:\users\mtopper\programming\python\git\dtocean-core\dtocean_core\interfaces\moorings.py:1119: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  economics_data["Quantity"].convert_objects(convert_numeric=True)


## Examine the results

Currently, there is no robustness built into the core, so the assumption is that the module executed successfully. This will have to be improved towards deployment of the final software.

Let's check the number of devices and annual output of the farm, using just information in the data object.

In [21]:
output_status = mooring_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [23]:
economics = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_foundations_economics_data")
economics.get_value(new_core, new_project)

,Key Identifier,Cost,Quantity,Year
0,id743,559148.667791,1.0,0.0
1,id472,1389.360000,1.0,0.0
2,id316,67.500000,1.0,0.0
3,id69,1015.154227,1.0,0.0
4,id316,67.500000,1.0,0.0
5,id422,2641.043412,1.0,0.0
6,id316,67.500000,1.0,0.0
7,id339,746.280000,1.0,0.0
8,id316,67.500000,1.0,0.0
9,id472,1389.360000,1.0,0.0


In [24]:
moorings = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_component_data")
moorings.get_value(new_core, new_project)

,Line Identifier,Marker
0,line000,0
1,line000,1
2,line000,2
3,line000,3
4,line000,4
5,line000,5
6,line000,6
7,line000,7
8,line001,8
9,line001,9


In [26]:
lines = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_line_data")
lines.get_value(new_core, new_project)

,Line Identifier,Type,Length,Dry Mass
0,line000,catenary,219.174306,1507.174176
1,line001,catenary,220.467777,1490.406841
2,line002,catenary,212.340890,1600.559112
3,line003,catenary,205.518546,1703.238915


In [27]:
umbilicals = mooring_branch.get_output_variable(new_core, new_project, "project.umbilical_cable_data")
umbilicals.get_value(new_core, new_project)

,Key Identifier,Length,Dry Mass,Required Floatation,Floatation Length,Marker
0,id743,931.914446,8107.655683,119.444997,186.382889,0032


In [28]:
dimensions = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_dimensions")
dimensions.get_value(new_core, new_project)

,System Identifier,Footprint Area,Volume
0,subhub001,0.000000,0.000000e+00
1,device001,15707.963268,1.898742e+06
2,subhub002,0.000000,0.000000e+00
3,array,0.000000,0.000000e+00


In [29]:
network = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_foundations_network")
net_val = network.get_value(new_core, new_project)

In [30]:
import pprint
pprint.pprint(net_val)

{'nodes': {'array': {'Substation foundation': {'marker': [[41, 42]],
                                               'quantity': Counter({'id718': 1, 'grout': 1})}},
           'device001': {'Foundation': {'marker': [[33, 34],
                                                   [35, 36],
                                                   [37, 38],
                                                   [39, 40]],
                                        'quantity': Counter({'grout': 4, 'id516': 3, 'id528': 1})},
                         'Mooring system': {'marker': [[0,
                                                        1,
                                                        2,
                                                        3,
                                                        4,
                                                        5,
                                                        6,
                                                        7],
                 